In [1]:
DATABASE_PATH = '/media/ybahat/data/Datasets/Diversity'
DATASET_GROUP = 'webcams' #'faces'

In [2]:
import cv2
import os
from interactivecrop.interactivecrop import main as crop
import numpy as np

In [ ]:
# When using youtube-dl:
# VIDEO_URL = 'https://www.youtube.com/watch?v=I2uGOQTY9uk&t=1s'
VIDEO_URL = 'https://www.youtube.com/watch?v=HpZAez2oYsA'

import youtube_dl
ydl = youtube_dl.YoutubeDL({'outtmpl': '%(id)s.%(ext)s'})

with ydl:
    result = ydl.extract_info(
        VIDEO_URL,
        download=True # We just want to extract the info
    )

downloaded_file = '%s.%s'%(result['id'],result['ext'])

# An alternative downloading method, from terminal:
# streamlink --hls-live-edge 99999 --hls-segment-threads 5 --hls-duration 1:00:00 -o "video.mp4" https://www.youtube.com/watch?v=RQA5RcIZlAM best

# Alternatively, download by recording network streaming using VLC recorder. See instructions in this link:
# https://confluence.bethel.edu/display/ITSKB/Recording+a+Network+Stream+with+VLC+Player

In [3]:
# If not using youtube-dl:
downloaded_file = '/media/ybahat/data/Datasets/Diversity/webcams/video.mp4'
result = {'title':'VeneziaBeach'}

In [5]:
FRAME_NUM_4_CROPPING = 1100

vidcap = cv2.VideoCapture(downloaded_file)
vidcap.set(cv2.CAP_PROP_POS_FRAMES,FRAME_NUM_4_CROPPING)
success,image = vidcap.read()
# cropped_shape = crop([image[...,[2,1,0]]],['temp'],optimize=False)
crop([image[...,[2,1,0]]],optimize=False)
# image.shape

GridBox(children=(Dropdown(description='Img Name:', layout=Layout(grid_area='im_selector', width='100%'), opti…

In [6]:
cropping_shape = (0,265, 1920,783)
temp_shape = list(cropping_shape)
temp_shape[2] = int(np.ceil(cropping_shape[2]/16)*16)
temp_shape[3] = int(np.ceil(cropping_shape[3]/16)*16)
temp_shape[0] += (cropping_shape[2]-temp_shape[2])//2
# temp_shape[1] -= (cropping_shape[3]-temp_shape[3])//2
cropping_shape = tuple(temp_shape)
print(cropping_shape)

(0, 265, 1920, 784)


In [7]:
CLEANUP_SCALE_FACTOR = 2

output_folder = os.path.join(DATABASE_PATH,DATASET_GROUP,result['title'])
os.mkdir(output_folder)
output_folder = os.path.join(output_folder,'raw_frames' if CLEANUP_SCALE_FACTOR==1 else 'clean_frames')
os.mkdir(output_folder)

In [8]:
vidcap.set(cv2.CAP_PROP_POS_FRAMES,0)
success,image = vidcap.read()
count = 1
while success:
    if count>300000:
        break
    image = image[cropping_shape[1]:cropping_shape[1]+cropping_shape[3],cropping_shape[0]:cropping_shape[0]+cropping_shape[2],...]
    if CLEANUP_SCALE_FACTOR>1:
        image = cv2.resize(image,dsize=(cropping_shape[2]//CLEANUP_SCALE_FACTOR,cropping_shape[3]//CLEANUP_SCALE_FACTOR),interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(os.path.join(output_folder,"frame%d.png" % count),image)
    count += 1
    success,image = vidcap.read()


print('Saved %d frames'%(count))

Saved 300001 frames


In [9]:
os.remove(downloaded_file)
